<a href="https://colab.research.google.com/github/ellerywuyn/LEAR-lab/blob/3-stratification-and-percent-offensive/profanity_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Offensive Language Detection

## Part 1: Calculate the average F1, R and P for 4 stratified samples
## Compare transformer vs non-transformer model


### Transformer Setups

In [2]:
# import packages
!pip install transformers
import shutil
import time
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import csv
import urllib.request

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.7 MB/s eta 0:00:00


In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
# remove local model repo
shutil.rmtree('cardiffnlp', ignore_errors=True)

In [6]:
# preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

task='offensive'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# initiate model
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

### Read in twitter dataset

In [7]:
tweets_df = pd.read_csv("/content/drive/MyDrive/labeled_data.csv")

In [8]:
tweets_df.sample(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
10863,11145,3,0,3,0,1,I saw so many basic bitches just picking up my...
14871,15227,9,0,9,0,1,RT @DJZeeti: i couldnt host 106 and park .. i'...
12685,13000,3,0,3,0,1,"Luda said that his bitch bad, well mine looks ..."
22478,22953,3,0,0,3,2,Was finna slit my eyebrows up in the shop but ...
19798,20238,6,0,6,0,1,RT @olmelonhead: when bitches spread rumors ab...
23239,23723,3,0,3,0,1,You always find something to bitch about.
4936,5082,3,0,0,3,2,@TheMayorMatt @Frizzle18 I like where ur domes...
13356,13684,3,0,3,0,1,Niggas who stand in line for Jays mad at me no...
23247,23731,3,0,2,1,1,You are beyond retarded
3296,3382,3,0,3,0,1,@GossipCop they are all idiots except kourt......


### Data Wrangling

In [9]:
# subset variables of interest
tweets_df_interest = tweets_df[["class", "tweet"]]

In [10]:
# remove the hate speech rows
tweets_df_interest = tweets_df_interest[tweets_df_interest["class"].isin([1,2])]

In [11]:
tweets_df_interest.shape

(23353, 2)

In [12]:
# get the # of rows for class == 2 because it has significant less rows than class == 1 
# we want to match them by undersampling class == 1
n_sample = sum(tweets_df_interest["class"] == 2)
n_sample

4163

### Sampling and Fitting

In [13]:
# define a function to generate labels using the transformer model
def get_label(input):
  text = input
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  # scores[0] is the not-offensive score
  # not-offensive score <= 0.5 -> label 1
  # not-offensive score > 0.5 -> label 0
  return 1 if scores[0] <= 0.5 else 0

In [14]:
# universal assignments
seed = [1, 20, 66, 123]
target_names = ["not offensive", "offensive"]
class2 = tweets_df_interest["class"][tweets_df_interest["class"] == 2].index

In [15]:
accuracy, precision, recall, f1 = [], [], [], []

for i in range(4):
  # randomly sample n_sample rows from class == 1
  class1 = tweets_df_interest["class"][tweets_df_interest["class"] == 1].sample(n_sample, random_state = seed[i]).index 

  # combine with class == 2
  sampled_tweets_df = tweets_df_interest.loc[class1.union(class2)]

  # change all the class == 2 (non-offensive) to 0 to conform to convention
  sampled_tweets_df["class"] = sampled_tweets_df["class"].replace(2, 0)

  # use the model to get predictions and store them in a new column named "predictions" in sampled_tweets_df
  sampled_tweets_df['prediction'] = sampled_tweets_df.apply(lambda x: get_label(x["tweet"]), axis=1)

  # change the "class" column to "label" for clarity purposes
  sampled_tweets_df = sampled_tweets_df.rename(columns = {"class": "label"})

  # reorder the columns
  sampled_tweets_df = sampled_tweets_df.loc[:,['tweet','label','prediction']]

  # print the classification reports for all 4
  print(classification_report(sampled_tweets_df["label"], sampled_tweets_df["prediction"], target_names=target_names))

  # append to 4 metrics
  accuracy.append(accuracy_score(sampled_tweets_df["label"], sampled_tweets_df["prediction"]))
  precision.append(precision_score(sampled_tweets_df["label"], sampled_tweets_df["prediction"]))
  recall.append(recall_score(sampled_tweets_df["label"], sampled_tweets_df["prediction"]))
  f1.append(f1_score(sampled_tweets_df["label"], sampled_tweets_df["prediction"]))

               precision    recall  f1-score   support

not offensive       0.94      0.75      0.84      4163
    offensive       0.79      0.96      0.87      4163

     accuracy                           0.85      8326
    macro avg       0.87      0.85      0.85      8326
 weighted avg       0.87      0.85      0.85      8326

               precision    recall  f1-score   support

not offensive       0.94      0.75      0.84      4163
    offensive       0.79      0.96      0.87      4163

     accuracy                           0.85      8326
    macro avg       0.87      0.85      0.85      8326
 weighted avg       0.87      0.85      0.85      8326

               precision    recall  f1-score   support

not offensive       0.95      0.75      0.84      4163
    offensive       0.79      0.96      0.87      4163

     accuracy                           0.85      8326
    macro avg       0.87      0.85      0.85      8326
 weighted avg       0.87      0.85      0.85      8326

 

In [16]:
metrics = {
    "Metrics": ["Avg Accuracy", "Avg Precision", "Avg Recall", "Avg F1"],
    "Values": [round(np.average(accuracy), 2), round(np.average(precision), 2), round(np.average(recall), 2), round(np.average(f1), 2)]
}
pd.DataFrame(metrics)

,Metrics,Values
0,Avg Accuracy,0.85
1,Avg Precision,0.79
2,Avg Recall,0.96
3,Avg F1,0.87


### Non-Transformer

In [19]:
!pip install profanity-filter
from profanity_filter import ProfanityFilter
!python -m spacy download en
pf = ProfanityFilter()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_web_sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047102 sha256=3b3c10b2c7f0bb6014b00b1f8be27897e9198759011a7e539c7d6461cf72c9e8
  Stored in directory: /root/.cache/pip/wheels/19/d6/1c/5484b95647df5d7afaf74abde458c66c1cd427e69e801fe826
Successfully built en_core_web_sm
  Attempting uninstall: en_core_web_sm
    Found existing installation: en-core-web-sm 3.5.0
    Uninstalling en-core-web-sm-3.5.0:
      Successfully uninstalled en-core-web-sm-3.5.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.9/dist-packages/en_core_web_sm -->
/usr/loca

In [23]:
# define a function to generate labels using the transformer model
def get_label_n(input):
  return 1 if pf.is_profane(input) else 0

In [24]:
accuracy_n, precision_n, recall_n, f1_n = [], [], [], []

for i in range(4):
  # randomly sample n_sample rows from class == 1
  class1 = tweets_df_interest["class"][tweets_df_interest["class"] == 1].sample(n_sample, random_state = seed[i]).index 

  # combine with class == 2
  sampled_tweets_df = tweets_df_interest.loc[class1.union(class2)]

  # change all the class == 2 (non-offensive) to 0 to conform to convention
  sampled_tweets_df["class"] = sampled_tweets_df["class"].replace(2, 0)

  # use the model to get predictions and store them in a new column named "predictions" in sampled_tweets_df
  sampled_tweets_df['prediction'] = sampled_tweets_df.apply(lambda x: get_label_n(x["tweet"]), axis=1)

  # change the "class" column to "label" for clarity purposes
  sampled_tweets_df = sampled_tweets_df.rename(columns = {"class": "label"})

  # reorder the columns
  sampled_tweets_df = sampled_tweets_df.loc[:,['tweet','label','prediction']]
  sampled_tweets_df.sample(10)

  # print the classification reports for all 4
  print(classification_report(sampled_tweets_df["label"], sampled_tweets_df["prediction"], target_names=target_names))

  # append to 4 metrics
  accuracy_n.append(accuracy_score(sampled_tweets_df["label"], sampled_tweets_df["prediction"]))
  precision_n.append(precision_score(sampled_tweets_df["label"], sampled_tweets_df["prediction"]))
  recall_n.append(recall_score(sampled_tweets_df["label"], sampled_tweets_df["prediction"]))
  f1_n.append(f1_score(sampled_tweets_df["label"], sampled_tweets_df["prediction"])) 

               precision    recall  f1-score   support

not offensive       0.82      0.96      0.88      4163
    offensive       0.95      0.79      0.86      4163

     accuracy                           0.87      8326
    macro avg       0.88      0.87      0.87      8326
 weighted avg       0.88      0.87      0.87      8326

               precision    recall  f1-score   support

not offensive       0.83      0.96      0.89      4163
    offensive       0.95      0.80      0.87      4163

     accuracy                           0.88      8326
    macro avg       0.89      0.88      0.88      8326
 weighted avg       0.89      0.88      0.88      8326

               precision    recall  f1-score   support

not offensive       0.82      0.96      0.89      4163
    offensive       0.95      0.80      0.87      4163

     accuracy                           0.88      8326
    macro avg       0.89      0.88      0.88      8326
 weighted avg       0.89      0.88      0.88      8326

 

In [25]:
metrics = {
    "Metrics": ["Avg Accuracy", "Avg Precision", "Avg Recall", "Avg F1"],
    "Values": [round(np.average(accuracy_n), 2), round(np.average(precision_n), 2), round(np.average(recall_n), 2), round(np.average(f1_n), 2)]
}
pd.DataFrame(metrics)

,Metrics,Values
0,Avg Accuracy,0.88
1,Avg Precision,0.95
2,Avg Recall,0.80
3,Avg F1,0.86
